# BERT Test on PerSent

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install textattack[tensorflow]
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 7.6 MB/s 
     |████████████████████████████████| 5.5 MB 42.3 MB/s 
     |████████████████████████████████| 60 kB 8.4 MB/s 
     |████████████████████████████████| 401 kB 67.0 MB/s 
     |████████████████████████████████| 125 kB 79.4 MB/s 
     |████████████████████████████████| 41.4 MB 1.2 MB/s 
     |████████████████████████████████| 365 kB 67.0 MB/s 
     |████████████████████████████████| 3.6 MB 43.8 MB/s 
     |████████████████████████████████| 769 kB 51.7 MB/s 
     |████████████████████████████████| 511.7 MB 6.5 kB/s 
     |████████████████████████████████| 125 kB 78.5 MB/s 
     |████████████████████████████████| 5.8 MB 35.8 MB/s 
     |████████████████████████████████| 95 kB 5.9 MB/s 
     |████████████████████████████████| 182 kB 68.9 MB/s 
     |████████████████████████████████| 115 kB 61.7 MB/s 
     |█████████████████████████████

In [ ]:
import textattack

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data_path = "/content/gdrive/MyDrive/NLP_Project/dev_min.csv"

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv(data_path)
data.head(5)

,Unnamed: 0,DOCUMENT_INDEX,TITLE,TARGET_ENTITY,DOCUMENT,TRUE_SENTIMENT
0,0,3360,AS VALUES DROP MORE HOMEOWNERS WALKING AWAY FR...,Benjamin Koellmann,In 2006 Benjamin Koellmann bought a condomini...,Neutral
1,1,3361,Paraguay leader sacks chief of armed forces,Fernando Lugo,Lugo a former Catholic bishop who assumed off...,Positive
2,2,3362,Tennis: Nadal hopes to see Spain win World Cup,Rafael Nadal,Spanish Wimbledon winner Rafael Nadal said Sun...,Positive
3,3,3363,White House picks new cyber coordinator,Howard A. Schmidt,In a letter posted on the White House web site...,Positive
4,4,3364,MORRIS PICKS HIS QB BUT KEEPS IT QUIET,MORRIS,TAMPA At least Raheem Morris finally has the ...,Positive


In [ ]:
def preprocess_data(data):
    texts = data['DOCUMENT'].to_list()
    labels = []
    for sentiment in data['TRUE_SENTIMENT'].to_list():
        if sentiment == 'Positive':
            labels.append(2)
        elif sentiment == 'Negative':
            labels.append(0)
        else:
            labels.append(1)

    return texts, labels

data_texts, data_labels = preprocess_data(data)

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data_texts, data_labels, test_size=0.3)
val_texts, test_texts, val_labels, test_labels = train_test_split(data_texts, data_labels, test_size=0.5)

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', num_labels=3)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = '/content/gdrive/MyDrive/NLP_Project/results',
    num_train_epochs = 3,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir = '/content/gdrive/MyDrive/NLP_Project/logs',
    logging_steps = 10
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 404
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 78
  Number of trainable parameters = 109484547


Step,Training Loss
10,1.040500
20,1.065600
30,0.987600
40,0.988200
50,0.957500
60,1.021700
70,0.941100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=78, training_loss=0.9884389608334272, metrics={'train_runtime': 110.8567, 'train_samples_per_second': 10.933, 'train_steps_per_second': 0.704, 'total_flos': 318893462286336.0, 'train_loss': 0.9884389608334272, 'epoch': 3.0})

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 289
  Batch size = 64


PredictionOutput(predictions=array([[-7.97382236e-01,  1.57364100e-01,  2.99225837e-01],
       [-7.56357431e-01,  1.21793851e-01,  1.50109485e-01],
       [-9.24351037e-01, -1.75991785e-02,  4.40750331e-01],
       [-8.08160901e-01,  9.44373682e-02,  4.58889544e-01],
       [-8.60078931e-01,  1.44062966e-01,  5.84984601e-01],
       [-9.91909444e-01,  2.49504730e-01,  4.05932456e-01],
       [-9.05212045e-01,  1.40529573e-01,  3.32735866e-01],
       [-8.41530561e-01,  1.69074386e-01,  5.17834961e-01],
       [-8.26542556e-01, -1.55477881e-01,  6.24275386e-01],
       [-8.39629710e-01,  1.27622515e-01,  4.44206327e-01],
       [-7.95637965e-01,  2.93099973e-02,  3.96342188e-01],
       [-8.65271151e-01,  3.76494490e-02,  5.01491189e-01],
       [-9.27324414e-01,  1.75613686e-01,  5.21261394e-01],
       [-8.63783479e-01,  7.87816048e-02,  4.10000265e-01],
       [-8.93924594e-01,  1.14454655e-02,  6.30095661e-01],
       [-9.13826883e-01,  1.33607537e-01,  5.95052600e-01],
       [-9.